# **日本国内におけるSentinel-1のアーカイブデータと、近傍の気象台・測候所における撮像日の気象データをリスト化します**

# 1.使用ライブラリの読み込み

In [14]:
import os
import requests
import json
import pprint
import pandas as pd
import folium 
import geopandas as gpd
import sys
import datetime
import csv
from tkinter import filedialog
import numpy as np
import urllib3
from urllib3.exceptions import InsecureRequestWarning
from collections import OrderedDict
import geojson
from shapely.geometry import shape
import time
from shapely.wkt import loads
from shapely.geometry import shape
import geopandas as gpd

import matplotlib.pyplot as plt
from functools import partial
import pyproj
from shapely.ops import transform
import ipywidgets as widgets

import asf_search as asf

urllib3.disable_warnings(InsecureRequestWarning) #API参照時に出てくるWarningを非表示にする一文

使用する関数の読み込み

In [15]:
def get_any_geojson_file(foldr_path):
    for file in os.listdir(folder_path):
        if file.endswith('.geojson'):
            return os.path.join(folder_path, file)
        return None    

# 2.geojsonの読み込み
*AOIフォルダにGeojsonを置いた上で実行してください。

使用している気象庁の過去データを引っ張ってくるAPIは「気象台・測候所等」の地点のみしか対応していない点に留意

地点名 地点番号

- 北海道
    - 稚内 47401, 北見枝幸 47402, 羽幌 47404, 雄武	47405, 留萌 47406, 旭川	47407, 網走 47409, 小樽	47411, 札幌 47412, 岩見沢 47413, 帯広 47417, 釧路 47418, 根室 47420, 寿都 47421, 室蘭 47423, 苫小牧 47424, 浦河 47426, 江差 47428, 函館 47430, 倶知安 47433, 紋別 47435, 広尾 47440
- 青森
    - 深浦 47574, 青森 47575, むつ 47576,	八戸 47581
- 岩手
    - 大船渡 47512, 盛岡	47584, 宮古 47585
- 秋田
    - 秋田 47582
- 宮城
    - 石巻 47592, 仙台 47590
- 山形
    - 山形 47588, 新庄 47520, 酒田 47587
- 福島
    - 福島 47595, 白河 47597, 若松 47570, 小名浜	47598
<br><br>
- 茨木
    - 水戸 47629, つくば（館野）47646
- 栃木
    - 奥日光 47690, 宇都宮 47615
- 埼玉
    - 熊谷 47626, 秩父 47641
- 群馬
    - 前橋 47624
- 東京
    - 東京 47662, 大島 47675, 八丈島 47678, 三宅島 47677, 父島 47971, 南鳥島 47991 
- 千葉
    - 館山 47672, 千葉 47682, 銚子 47648, 勝浦 47674
- 神奈川
    - 横浜 47670
<br><br>
- 新潟
    - 相川 47602, 新潟 47604, 高田 47612, 
- 富山
    - 伏木 47606, 富山 47607
- 石川
    - 輪島 47600, 金沢 47605
- 福井
    - 福井 47616, 敦賀 47631
- 山梨
    - 甲府 47638, 河口湖	47640
- 長野
    - 長野 47610, 軽井沢	47622, 松本 47618, 飯田	47637, 諏訪 47620
- 岐阜
    - 高山 47617, 岐阜 47632
- 静岡
    - 富士山 47639, 三島	47657, 網代 47668, 石廊崎 47666, 静岡 47656, 御前崎 47655, 浜松	47654
- 愛知
    - 名古屋 47636, 伊良湖 47653
<br><br>
- 三重
    - 上野 47649, 津 47651, 尾鷲 47663, 四日市 47684
- 滋賀
    - 伊吹山 47751, 彦根	47761
- 京都
    - 舞鶴 47750, 京都 47759
- 大阪
    - 大阪 47772
- 兵庫
    - 豊岡 47747, 姫路 47769, 神戸 47770, 洲本 47776
- 奈良
    - 奈良 47780
- 和歌山
    - 和歌山 47777, 潮岬	47778
<br><br>
- 鳥取
    - 鳥取 47746, 米子 47744, 境	47742
- 島根
    - 松江 47741, 浜田 47755, 西郷 47740
- 岡山
    - 津山 47756, 岡山 47768
- 広島
    - 福山 47767, 広島 47765, 呉	47766
- 山口
    - 下関 47762, 萩 47754, 山口	47784
- 徳島
    - 徳島 47895, 剣山 47894
- 香川
    - 高松 47891, 多度津	47890
- 愛媛
    - 松山 47887, 宇和島	47892
- 高知
    - 室戸岬 47899, 清水	47898, 宿毛 47897, 高知 47893
<br><br>
- 福岡
    - 飯塚 47809, 福岡	47807
- 佐賀
    - 佐賀 47813
- 長崎
    - 平戸 47805, 厳原 47800, 長崎 47817, 福江 47843, 佐世保 47812, 雲仙岳 47818
- 熊本
    - 阿蘇山 47821, 熊本	47819, 牛深 47838, 人吉	47824
- 大分
    - 日田 47814, 大分 47815
- 宮崎
    - 延岡 47822, 都城 47829, 宮崎 47830, 油津 47835
- 鹿児島
    - 阿久根 47823, 鹿児島 47827, 枕崎 47831, 種子島 47837, 屋久島 47836, 名瀬 47909, 沖永良部 47942
- 沖縄
    - 久米島 47929, 名護	47940, 那覇 47936, 南大東（南大東島） 47945, 与那国島 47912, 石垣島 47918, 宮古島 47927

参考：https://www.data.jma.go.jp/stats/etrn/index.php?prec_no=43&block_no=&year=&month=&day=&view=

In [23]:
area = '47600' #気象データを参照するのに使用。スクリプト上部をみて地点番号を入力してください。

folder_path = 'AOI'

filename = get_any_geojson_file(folder_path)

print(filename)

df_gjs = gpd.read_file(filename,encoding='cp932') #geopandasを使用してgeojsonを開く
ftr = df_gjs.iloc[0] #最初のフィーチャを選択
geom = ftr.geometry #ジオメトリを取得、WKT形式で取得できる
p = df_gjs.centroid #ポリゴンの重心取得
    
location = [p.y[0],p.x[0]] #重心座標のX,Yを抽出

m = folium.Map(location, zoom_start=13) #地図で表示
folium.GeoJson(str(filename)).add_to(m)
m

AOI\TEST.geojson


C:\Users\Shingo Yoshino\AppData\Local\Temp\ipykernel_2892\2623366295.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  p = df_gjs.centroid #ポリゴンの重心取得


AOIの面積算出

In [24]:
#アップロードしたAOIのジオメトリ化と面積算出

geom1 = str(geom)
area_aoi = geom.area

print(area_aoi)

0.0026146662570698505


# 3.アーカイブの検索
検索条件を設定してください。
<br>参照：衛星画像検索「Sentinelsat」 https://sentinelsat.readthedocs.io/en/stable/

In [25]:
#検索条件
results = asf.geo_search(
    platform=[asf.PLATFORM.SENTINEL1],
    intersectsWith=geom1,
    #flightDirection=asf.FLIGHT_DIRECTION.ASCENDING,
    processingLevel=asf.PRODUCT_TYPE.GRD_HD, #GRD_HD,SLC
    start='2024-04-01T00:00:00Z',
    end='2024-06-28T00:00:00Z'    
)

#ASFSearchResultsオブジェクトをgeojsonに変換
data = results.geojson()

#正規化表現
team_df = pd.json_normalize(data, record_path='features')

In [19]:
#WKTの属性枠を追加
WKT_Ary = []
CoverRate_Ary = []

                  
#WKTの属性を追加
for i in range(len(team_df)):
    
    geometry2 = team_df.iloc[i]['geometry.coordinates'] #正規化表現内からジオメトリを抜き出し
    
    #ジオメトリを文字列のWKT方式に変換
    wkt = 'POLYGON((' + str(geometry2[0][0][0]) + ' ' + str(geometry2[0][0][1]) + ', ' + \
        str(geometry2[0][1][0]) + ' ' + str(geometry2[0][1][1]) + ', ' + \
        str(geometry2[0][2][0]) + ' ' + str(geometry2[0][2][1]) + ', ' + \
        str(geometry2[0][3][0]) + ' ' + str(geometry2[0][3][1]) + ', ' + \
        str(geometry2[0][4][0]) + ' ' + str(geometry2[0][4][1]) + '))'
    
    geom2 = loads(wkt) #文字列WKTにジオメトリとしての情報を付加
    
    intersection_area =  geom2.intersection(geom).area #交差部の面積を演算
    cover_rate = intersection_area / area_aoi #AOI面積と交差部面積の比率演算
    
    #配列に演算結果を追加
    WKT_Ary.append(wkt)
    CoverRate_Ary.append(cover_rate)

#辞書の中に配列を追加    
team_df['WKT'] = WKT_Ary
team_df['Coverage'] = CoverRate_Ary

#辞書から不要な要素を消去
del team_df['type'],team_df['geometry.coordinates'],team_df['geometry.type'],\
    team_df['properties.browse'],team_df['properties.bytes'],team_df['properties.centerLat'],\
    team_df['properties.centerLon'],team_df['properties.faradayRotation'],team_df['properties.groupID'],\
    team_df['properties.granuleType'],team_df['properties.insarStackId'],team_df['properties.md5sum'],\
    team_df['properties.offNadirAngle'],team_df['properties.orbit'],team_df['properties.pointingAngle'],\
    team_df['properties.sceneName'],team_df['properties.sensor'],team_df['properties.startTime'],\
    team_df['properties.stopTime'],team_df['properties.pgeVersion'],team_df['properties.fileName']

print(team_df)

  properties.beamModeType                                  properties.fileID  \
0                      IW  S1A_IW_GRDH_1SDV_20240617T204330_20240617T2043...   
1                      IW  S1A_IW_GRDH_1SDV_20240605T204331_20240605T2043...   
2                      IW  S1A_IW_GRDH_1SDV_20240524T204331_20240524T2043...   
3                      IW  S1A_IW_GRDH_1SDV_20240512T204331_20240512T2043...   
4                      IW  S1A_IW_GRDH_1SDV_20240430T204332_20240430T2043...   
5                      IW  S1A_IW_GRDH_1SDV_20240418T204331_20240418T2043...   
6                      IW  S1A_IW_GRDH_1SDV_20240406T204331_20240406T2043...   

  properties.flightDirection  properties.pathNumber properties.platform  \
0                 DESCENDING                     46         Sentinel-1A   
1                 DESCENDING                     46         Sentinel-1A   
2                 DESCENDING                     46         Sentinel-1A   
3                 DESCENDING                     46        

# 4.CSVとして出力
気象データ不要の場合はココまで

In [18]:
#カレントディレクトリへCSV出力

team_df.to_csv('./Sentinel-1_List.csv')

# 5.気象データの検索
上記で呼び出した撮像日と、「気象観測所・測候所」の地点番号に基づいて、過去の気象データを呼び出します。
<br>参照：気象庁過去データ https://www.nakakamado.com/2022/09/weather-past-webapi.html

使用する関数の読み込み

In [102]:
#気象データ呼び出し関数

def api_call(area,dt):
    url = 'https://api.cultivationdata.net/past?no=' + str(area) + '&year=' + str(dt.year) + '&month=' + str(dt.month)
    r = requests.get(url, verify=False)
    data = json.loads(r.text)
    return data

#気象データ検索関数

def api_search(data,date,key):
    for val in data.values():
        result = str(val[date][key])
    return result

気象データの検索
<br>*APIの呼び出しに少し時間がかかります。

In [103]:
#WKTの属性枠を追加
total_rainfall_Ary = []
total_snowfall_Ary = []
weather_overview_day_y_Ary = []
weather_overview_night_y_Ary = []
weather_overview_day_Ary = []
weather_overview_night_Ary = []

for i , row in team_df.iterrows():
    
    s = pd.to_datetime(row[6])
    dt = s.tz_convert('Asia/Tokyo')
    print(dt)
    
    day = datetime.timedelta(days = 1)
    dt_y = dt - day
        
    data = api_call(area,dt)
    data_y = api_call(area,dt_y)
    
    time.sleep(2)
        
    year = dt.year
    month = dt.month
    day = dt.day
        
    year_y = dt_y.year
    month_y = dt_y.month
    day_y = dt_y.day
        
    date = str(year) + '-' + str(month) + '-' + str(day)
    date_y = str(year_y) + '-' + str(month_y) + '-' + str(day_y)
    
    total_rainfall_Ary.append(api_search(data,date,'合計降水量'))
    total_snowfall_Ary.append(api_search(data,date,'合計降雪'))
    weather_overview_day_y_Ary.append(api_search(data_y,date_y,'天気概況（昼）'))
    weather_overview_night_y_Ary.append(api_search(data_y,date_y,'天気概況（夜）'))
    weather_overview_day_Ary.append(api_search(data,date,'天気概況（昼）'))
    weather_overview_night_Ary.append(api_search(data,date,'天気概況（夜）'))
    
team_df['合計降水量'] = total_rainfall_Ary
team_df['合計降雪'] = total_snowfall_Ary
team_df['前日の天気概況（昼）'] = weather_overview_day_y_Ary
team_df['前日の天気概況（夜）'] = weather_overview_night_y_Ary
team_df['当日の天気概況（昼）'] = weather_overview_day_Ary
team_df['当日の天気概況（夜）'] = weather_overview_night_Ary

2023-05-19 05:43:28+09:00
2023-05-07 05:43:27+09:00
2023-04-25 05:43:26+09:00
2023-04-13 05:43:26+09:00
2023-04-01 05:43:25+09:00
2023-03-20 05:43:25+09:00
2023-03-08 05:43:25+09:00


# 6.CSVとして出力

In [104]:
#カレントディレクトリへCSV出力

team_df.to_csv('./Sentinel-1_List.csv', encoding="shift-jis")

# A.データフレームの確認

In [170]:
#確認用
print(team_df)

  properties.beamModeType                                  properties.fileID  \
0                      IW  S1A_IW_GRDH_1SDV_20240430T204332_20240430T2043...   
1                      IW  S1A_IW_GRDH_1SDV_20240418T204331_20240418T2043...   
2                      IW  S1A_IW_GRDH_1SDV_20240406T204331_20240406T2043...   
3                      IW  S1A_IW_GRDH_1SDV_20240325T204330_20240325T2043...   
4                      IW  S1A_IW_GRDH_1SDV_20240313T204330_20240313T2043...   
5                      IW  S1A_IW_GRDH_1SDV_20240301T204330_20240301T2043...   

  properties.flightDirection  properties.pathNumber properties.platform  \
0                 DESCENDING                     46         Sentinel-1A   
1                 DESCENDING                     46         Sentinel-1A   
2                 DESCENDING                     46         Sentinel-1A   
3                 DESCENDING                     46         Sentinel-1A   
4                 DESCENDING                     46         Sent